In [27]:
import pandas as pd
import numpy as np
from csv import reader
from numpy import linalg as la
from sys import exit
from haversine import haversine
import math
from operator import itemgetter

#시각화 도구
import folium
from folium.features import DivIcon
import random
import matplotlib.pyplot as plt

# 전역 변수 설정
FILE_SAVE_DIR = './Data_set_0402/' #있다면 추가 해주기 (상대경로)
line_color_list = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                    'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
                    'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'] 
# Global Variable Setting

# Nicname = '5CAC8432AE4EC54B'
# Huzzi = '44754E5A077F32B4'
# oizi = '2DDB3706DE4F7B45'
kyounghwan = '3963650B38CAF112'
ryeong = '72294652E731441B'
soo = '29D97405E68FA578'

PATIENT_UID = kyounghwan # 감염자 UID
TARGET_UID =  ryeong # 조사 대상자 UID

# Map Setting
Raw_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
Filter_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
mass_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
compare_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사

#Call CSV file
def load_GPS_set(UID): # Data load
    try:
        data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
        data_set['TIME STAMP'] = pd.to_datetime(data_set['LOG TIME']).dt.strftime('%Y-%m-%d %H:%M:%s')
        # data_set['TIME'] = data_set['TIME STAMP'].dt.strftime('%Y-%m-%d %H:%M') # 초단위 자르기
        # data_set['STAMP_TIME'] = pd.to_datetime(data_set['TIME']) # stamp_time datetime 형식으로 변경
        re_data_set = data_set[["TIME STAMP","UID","altitude","latitude","longitude","provider"]]
        return re_data_set
    
    except FileNotFoundError as e:
        pass
def load_Sateillate_set(UID): # Data load
    try:
        data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_sate"+".csv", sep=",", dtype='unicode')
        
        data_set['TIME STAMP'] = pd.to_datetime(data_set['LOG TIME']).dt.strftime('%Y-%m-%d %H:%M:%s')
        # data_set['TIME'] = data_set['TIME STAMP'].dt.strftime('%Y-%m-%d %H:%M') # 초단위 자르기
        # data_set['STAMP_TIME'] = pd.to_datetime(data_set['TIME']) # stamp_time datetime 형식으로 변경
        re_data_set = data_set[["TIME STAMP","UID","SNR AVERAGE","SATELLITE COUNT"]]
        return re_data_set
    
    except FileNotFoundError as e:
        pass

def merge_gps_sati_set(UID): # for weight_filter
    gps_df = load_GPS_set(UID)
    sat_df = load_Sateillate_set(UID)
    merge_df = pd.merge(gps_df,sat_df, how='left', on = ['TIME STAMP','UID'])
    return merge_df

################### RAW DATA FUNC ######################
def extract_raw_data(UID,start,end):
    Dataframe = merge_gps_sati_set(UID)
    marker_list = []
    for count in range(start,end):
        marker_point_lati = float(Dataframe.loc[count,'latitude'])
        marker_point_long = float(Dataframe.loc[count,'longitude'])
        marker_point_time = Dataframe.loc[count,'TIME STAMP']
        marker_point = [marker_point_time, marker_point_lati, marker_point_long]
        marker_list.append(marker_point)
    return marker_list
    
################### ABOUT MAP DROWING FUNC #####################
def draw_Polyline_map(map, marker_list, draw_color):
    replace_list = []
    for marker_point in marker_list:
        replace_list.append(marker_point[1:3])
    folium.PolyLine(locations=replace_list,tooltip='Polyline',color=draw_color).add_to(map)

def draw_index_map(map, marker_list):
    count = len(marker_list)
    for marker_point in marker_list:
        folium.Marker(marker_point[1:3], icon=DivIcon(icon_size=(10,3),icon_anchor=(7,20),
        html='<div style="font-size: 10pt; color : black">'+str(count)+'</div>',
        )).add_to(map)
        count = count - 1

def draw_circle_map(map, marker_list,draw_radius,draw_color):
    for marker_point in marker_list:
        folium.CircleMarker(location=marker_point[1:3], radius=draw_radius, color= draw_color).add_to(map)
        
def draw_arrow_map(map, start_list, end_list, line_color, dot_color):
    for start_point in start_list:
        for end_point in end_list:
            if(start_point[0]==end_point[0]):
                try:
                    rot_x = end_point[1] - start_point[1]
                    rot_y = end_point[2] - start_point[2]
                    rot = math.acos(haversine([start_point[1],0],[end_point[1],0])/haversine(start_point[1:3],end_point[1:3]))
                    folium.CircleMarker(location=start_point[1:3], radius=3, color= dot_color).add_to(map)
                    folium.CircleMarker(location=end_point[1:3], radius=3, color= dot_color).add_to(map)
                    folium.PolyLine(locations=[start_point[1:3],end_point[1:3]],tooltip='Polyline',color=line_color).add_to(map)
                    folium.RegularPolygonMarker(end_point[1:3], fill_color='blue', number_of_sides=3, radius=5, rotation=rot).add_to(map)
                except IndexError :
                    break
                except ZeroDivisionError:
                    break


In [14]:
soo_raw = extract_raw_data(soo,1,517)
draw_Polyline_map(Raw_map,soo_raw,line_color_list[1])
# draw_index_map(Raw_map,joung_raw)

ryeong_raw = extract_raw_data(ryeong,1,314)
draw_Polyline_map(Raw_map,ryeong_raw,line_color_list[0])

kyoung_raw = extract_raw_data(kyounghwan,1,516)
draw_Polyline_map(Raw_map,kyoung_raw,line_color_list[3])
# draw_index_map(Raw_map,kyoung_raw)

In [28]:
FILTER_CONSTANT = 5

soo_raw = extract_raw_data(soo,1,517)
avg_point = []

for idx in range(len(soo_raw)):
    x = soo_raw[idx][1]
    y = soo_raw[idx][2]
    avg_x = 0
    avg_y = 0
    for avgIdx in range(idx,idx + FILTER_CONSTANT):
        try:
            avg_x += soo_raw[avgIdx][1]
            avg_y += soo_raw[avgIdx][2]
        except IndexError:
            break
    avg_x = avg_x / FILTER_CONSTANT
    avg_y = avg_y / FILTER_CONSTANT
    point = [soo_raw[idx][0], avg_x, avg_y]
    avg_point.append(point)
draw_arrow_map(Filter_map,soo_raw,avg_point,"red", "blue")

Filter_map

ValueError: Location should consist of two numerical values, but [37.55043197978774, 127.07538485911748] of type <class 'list'> is not convertible to float.